In [1]:
import pandas as pd

In [2]:
RAW_FILE_PATH = "../data/raw/tsunami-events.tsv"
PROCESSED_FILE_PATH = "../data/processed/tsunami-events.csv"
COUNTRY_IDS_FILE_PATH = "../data/processed/world-110m-country-names.tsv"

### Data Cleaning

In [3]:
tsunami_df = pd.read_csv(RAW_FILE_PATH, sep="\t")
tsunami_df.head()

,Search Parameters,Year,Mo,Dy,Hr,Mn,Sec,Tsunami Event Validity,Tsunami Cause Code,Earthquake Magnitude,...,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,-2000.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,NaN,-1610.0,NaN,NaN,NaN,NaN,NaN,4.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
3,NaN,-1365.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,NaN,-1300.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tsunami_df.columns

Index(['Search Parameters', 'Year', 'Mo', 'Dy', 'Hr', 'Mn', 'Sec',
       'Tsunami Event Validity', 'Tsunami Cause Code', 'Earthquake Magnitude',
       'Vol', 'More Info', 'Deposits', 'Country', 'Location Name', 'Latitude',
       'Longitude', 'Maximum Water Height (m)', 'Number of Runups',
       'Tsunami Magnitude (Abe)', 'Tsunami Magnitude (Iida)',
       'Tsunami Intensity', 'Deaths', 'Death Description', 'Missing',
       'Missing Description', 'Injuries', 'Injuries Description',
       'Damage ($Mil)', 'Damage Description', 'Houses Destroyed',
       'Houses Destroyed Description', 'Houses Damaged',
       'Houses Damaged Description', 'Total Deaths', 'Total Death Description',
       'Total Missing', 'Total Missing Description', 'Total Injuries',
       'Total Injuries Description', 'Total Damage ($Mil)',
       'Total Damage Description', 'Total Houses Destroyed',
       'Total Houses Destroyed Description', 'Total Houses Damaged',
       'Total Houses Damaged Description'],
 

In [5]:
features_selected = [
    "Year",
    "Mo",
    "Tsunami Event Validity",
    "Tsunami Cause Code",
    "Earthquake Magnitude",
    "Country",
    "Location Name",
    "Latitude",
    "Longitude",
    "Tsunami Intensity",
    "Total Deaths",
    "Total Damage ($Mil)",
    "Total Houses Destroyed",
    "Total Injuries"
]

filtered_df = (
    tsunami_df
    .loc[tsunami_df["Year"] >= 1800, features_selected]
    .reset_index(drop=True)
)

In [6]:
filtered_df.columns = ["_".join(c.lower().split(" ")) for c in filtered_df.columns]
filtered_df = filtered_df.rename({
    "total_damage_($mil)": "total_damage_millions",
    "mo": "month"
}, axis=1)

In [7]:
filtered_df["month"] = pd.to_datetime(filtered_df['month'], format='%m').dt.month_name()
filtered_df["year"] = filtered_df["year"].astype(int)


In [8]:
filtered_df.head()

,year,month,tsunami_event_validity,tsunami_cause_code,earthquake_magnitude,country,location_name,latitude,longitude,tsunami_intensity,total_deaths,total_damage_millions,total_houses_destroyed,total_injuries
0,1800,June,2.0,1.0,NaN,PORTUGAL,AZORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1802,January,2.0,1.0,NaN,CROATIA,BAKAR,45.3,14.4,3.0,NaN,NaN,NaN,NaN
2,1802,March,2.0,1.0,NaN,ANTIGUA AND BARBUDA,ANTIGUA ISLAND & ST. CHRISTOPHER,17.2,-62.4,NaN,NaN,NaN,NaN,NaN
3,1802,August,1.0,0.0,NaN,UK,SW ENGLAND,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1802,August,3.0,1.0,NaN,VENEZUELA,CUMANA,10.3,-64.5,NaN,NaN,NaN,NaN,NaN


In [9]:
filtered_df = filtered_df[(filtered_df["tsunami_event_validity"] == 4) &
                          (filtered_df["tsunami_cause_code"] == 1)]

replace_country = {
    "RUSSIA": "Russian Federation",
    "USA": "United States",
    "IRAN": "Iran, Islamic Republic of",
    "MYANMAR (BURMA)": "Myanmar",
    "SOUTH KOREA": "Korea, Republic of",
    "TAIWAN": "Taiwan, Province of China",
    "VENEZUELA": "Venezuela, Bolivarian Republic of"
}

filtered_df["country"] = (
    filtered_df["country"]
    .replace(replace_country)
    .str.title()
    .str.replace("Of", "of")
)

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681 entries, 7 to 2166
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    681 non-null    int64  
 1   month                   679 non-null    object 
 2   tsunami_event_validity  681 non-null    float64
 3   tsunami_cause_code      681 non-null    float64
 4   earthquake_magnitude    668 non-null    float64
 5   country                 681 non-null    object 
 6   location_name           681 non-null    object 
 7   latitude                679 non-null    float64
 8   longitude               679 non-null    float64
 9   tsunami_intensity       307 non-null    float64
 10  total_deaths            233 non-null    float64
 11  total_damage_millions   85 non-null     float64
 12  total_houses_destroyed  121 non-null    float64
 13  total_injuries          178 non-null    float64
dtypes: float64(10), int64(1), object(3)
memor

In [10]:
country_ids = pd.read_csv(COUNTRY_IDS_FILE_PATH, sep="\t")
not_countries = set(filtered_df["country"]) - set(country_ids["name"])
filtered_df = filtered_df[~filtered_df["country"].isin(not_countries)]

filtered_df.to_csv(PROCESSED_FILE_PATH, index=False)

In [11]:
import altair as alt
from vega_datasets import data


world_map = alt.topo_feature(data.world_110m.url, 'countries')

events = filtered_df.groupby("country").size().reset_index(name='count')
events = events.merge(country_ids, how="right", left_on="country", right_on="name")
events["count"] = events["count"].fillna(0)

map = (
    alt.Chart(world_map)
    .mark_geoshape(stroke="grey", strokeWidth=0.3)
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(events, "id", ["name", "count"])
    )
    .encode(color=alt.condition("datum.count>0.0",
                                alt.Color("count:Q",
                                          scale=alt.Scale(scheme='blues',
                                                          domainMin=1,
                                                          domainMid=11),
                                          legend=alt.Legend(orient='bottom-left'),
                                          title="Total Tsunami Hits"),
                                alt.ColorValue("white")),
            tooltip=[alt.Tooltip("name:N", title="Country"),
                     alt.Tooltip("count:Q", title="Total Tsunami Hits")])
    .project("naturalEarth1")
    .properties(width=800, height=400)
)

filtered_df["color"] = "Tsunami Origin"

tsunami_spots = (
    alt.Chart(filtered_df)
    .mark_circle(size=5, color="red")
    .encode(
        latitude="latitude",
        longitude="longitude",
        color=alt.Color("color:N",
                        scale=alt.Scale(scheme="reds", domainMax=1),
                        legend=alt.Legend(title="", orient='top-left')),
        tooltip=[alt.Tooltip("earthquake_magnitude",
                 title="Earthquake Magnitude")]
    )
)

map + tsunami_spots

alt.LayerChart(...)